In [0]:
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import pipeline

In [0]:
import pandas as pd
import numpy as np

In [0]:
import nltk

## Loading The Dataset


In [6]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/cord-19/metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128492 entries, 0 to 128491
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   cord_uid          128492 non-null  object 
 1   sha               55751 non-null   object 
 2   source_x          128492 non-null  object 
 3   title             128464 non-null  object 
 4   doi               100586 non-null  object 
 5   pmcid             60771 non-null   object 
 6   pubmed_id         99124 non-null   float64
 7   license           128492 non-null  object 
 8   abstract          101611 non-null  object 
 9   publish_time      128477 non-null  object 
 10  authors           123725 non-null  object 
 11  journal           122195 non-null  object 
 12  mag_id            0 non-null       float64
 13  who_covidence_id  17071 non-null   object 
 14  arxiv_id          1395 non-null    object 
 15  pdf_json_files    55751 non-null   object 
 16  pmc_json_files    43

In [9]:
df.isnull().sum()

cord_uid                 0
sha                  72741
source_x                 0
title                   28
doi                  27906
pmcid                67721
pubmed_id            29368
license                  0
abstract             26881
publish_time            15
authors               4767
journal               6297
mag_id              128492
who_covidence_id    111421
arxiv_id            127097
pdf_json_files       72741
pmc_json_files       84739
url                  79319
s2_id                25512
dtype: int64

### I will we using only abstract to generate the summary.So filling the nan values in abstract columns.

In [0]:
df['abstract']=df['abstract'].fillna('no text')

In [13]:
df['abstract'][0]

'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were 

## Preprocessing Input for the Model

In [0]:
text=[]
for i in df['abstract']:
  text.append(i)

In [0]:
text=text[:1000]

In [0]:
str_text = ' '.join(text)

## Using BART Model

In [0]:
model = AutoModelWithLMHead.from_pretrained("bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("bart-large-cnn")

In [0]:
inputs = tokenizer.encode("summarize: " + str_text, return_tensors="pt", max_length=1024)
outputs = model.generate(inputs, max_length=200, min_length=100, early_stopping=True)

In [50]:
print([tokenizer.decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=False) for out in outputs][0])

40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. Cough (82.5%), fever (75%), and malaise (58.8%) were the most common symptoms. Most patients with pneumonia had crepitations (79.2%) but only 25% had bronchial breathing. Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress.


## Using T5 Model


In [51]:
model1 = AutoModelWithLMHead.from_pretrained("t5-base")
tokenizer1 = AutoTokenizer.from_pretrained("t5-base")

In [0]:
inputs = tokenizer1.encode("summarize: " + str_text, return_tensors="pt", max_length=512)
outputs = model1.generate(inputs, max_length=200, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)

In [55]:
print([tokenizer1.decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=False) for out in outputs][0])

this retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. most infections (92.5%) were community-acquired. infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%)
